In [3]:
from keras.models import Sequential
from keras.layers import Dense
import tensorflow as tf
import pandas as pd
import os
from sklearn import preprocessing
from sklearn.model_selection import train_test_split


In [5]:
root_directory = os.path.join(os.getcwd(), '../')
data_directory = os.path.join(root_directory ,'data')
pickle_name = 'After_preprocessing_raw_pickle'
pickle_path = os.path.join(data_directory, pickle_name)
raw_data = pd.read_pickle(pickle_path)
day_encoder = preprocessing.OneHotEncoder(sparse=False)
direction_encoder = preprocessing.OneHotEncoder(sparse=False)
raw_data

,Route,hour,Report Date,Year,Month,Daym,target,dir1,dir2,dir3,dir4,dir5,wd1,wd2,wd3,wd4,wd5,wd6,wd7
0,501,0,2014-01-02,2014,1,2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,501,0,2014-01-03,2014,1,3,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,501,0,2014-01-04,2014,1,4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,501,0,2014-01-05,2014,1,5,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,501,0,2014-01-06,2014,1,6,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3072885,310,23,2018-12-27,2018,12,27,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3072886,310,23,2018-12-28,2018,12,28,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3072887,310,23,2018-12-29,2018,12,29,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3072888,310,23,2018-12-30,2018,12,30,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [ ]:
raw_data = raw_data.drop(columns='Report Date')


In [ ]:
raw_data['Route'].nunique()
raw_data['Route'] = pd.Categorical(raw_data['Route'])
raw_data['Route'] = raw_data.Route.cat.codes

In [ ]:
target = raw_data.pop('target')

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(raw_data, target, test_size=0.3, random_state=31)

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((x_train.values, y_train.values))
test_dataset = tf.data.Dataset.from_tensor_slices((x_train.values, y_train.values))

In [ ]:
train_dataset = train_dataset.shuffle(len(train_dataset)).batch(1)

In [ ]:
def get_compiled_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(10, activation='relu'),
        tf.keras.layers.Dense(10, activation='relu'),
        tf.keras.layers.Dense(2)
    ])
    model.compile(optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), metrics=['accuracy'])
    return model

SyntaxError: invalid syntax (3848000348.py, line 6)

In [ ]:
model = get_compiled_model()


In [ ]:
model.fit(x_train, y_train, epochs= 1, batch_size=1000)

2152/2152 [==============================] - 4s 1ms/step - loss: 0.0803 - accuracy: 0.9825


922/922 [==============================] - 1s 853us/step


array([[-5.584856 ],
       [-2.9508548],
       [-2.9996355],
       ...,
       [-3.1125367],
       [-3.073001 ],
       [-7.0234776]], dtype=float32)

-2.9217386